# 0. Connect to a pre-provisioned Federation

In [1]:
from fast_api_client import Client, SyncOperations

client = Client(base_url="http://localhost:8000", verify_ssl=False, raise_on_unexpected_status=False, timeout=30.5)
operation = SyncOperations(client=client)

# 1. Aggregation Model Creation

In [2]:
from smart_broker_api import data_model_api

DATA_FRAME_NAME = "mvp_demo_data_frame"

data_model_id = data_model_api.new_data_model_data_frame(operation, DATA_FRAME_NAME)
data_model_api.data_frame_add_series(operation, data_model_id, "bmi_mean", "Observation:Body Mass Index", "AgregatorIntervalMean", "kg/m")
data_model_api.data_frame_add_series(operation, data_model_id, "bmi_first", "Observation:Body Mass Index", "AgregatorIntervalFirstOccurance", "kg/m")
data_model_api.data_frame_add_series(operation, data_model_id, "bmi_last", "Observation:Body Mass Index", "AgregatorIntervalLastOccurance", "kg/m")
data_model_tabular_id = data_model_api.new_tabular_model(operation)
data_model_tabular_id = data_model_api.tabular_model_add_dataframe_model(operation, data_model_tabular_id, data_model_id)

# 2. Flatten / Aggregate data from Aggregation model

In [3]:
from smart_broker_api import data_api

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"

longitudinal_id = data_api.read_longitudinal_fhirv1(operation)
tabular_dataset_id = data_api.read_dataset_tabular_from_longitudinal(operation, longitudinal_id, dataset_federation_id, "KCA Consortium", data_model_tabular_id)
data_frame_id = data_api.data_frame_tabular_select_data_frame(operation, tabular_dataset_id, DATA_FRAME_NAME)

# 3. Preprocessing: Remove NaNs from data frame

In [4]:
from smart_broker_api import preprocessing_api

no_nan_data_frame_id = preprocessing_api.dataframe_drop_missing(operation, data_frame_id)

# 4. Grab series to work with in Statistics Lib

In [5]:
series_1_id = data_api.data_frame_select_series(operation, no_nan_data_frame_id, "bmi_mean")
series_2_id = data_api.data_frame_select_series(operation, no_nan_data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

b2a41d2e-8ee7-4dbf-adc4-8d0d8d3bb85f , 821aeb6e-a601-4d3d-8159-92daa4615089


# 5. Cohort selection

In [6]:
from smart_broker_api import preprocessing_api
from smart_broker_api import statistics_api

query = "bmi_first > bmi_last"
query_dataframe = preprocessing_api.data_frame_query(operation, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(operation, query_dataframe, "bmi_last")

print(f"Patients whose BMI decreased during study: {statistics_api.count(operation,  query_series)}")

query = "bmi_first < bmi_last"
query_dataframe = preprocessing_api.data_frame_query(operation, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(operation, query_dataframe, "bmi_last")

print(f"Patients whose BMI increased during study: {statistics_api.count(operation,  query_series)}")

query = "bmi_first - bmi_last > 0.5"
query_dataframe = preprocessing_api.data_frame_query(operation, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(operation, query_dataframe, "bmi_last")

print(f"Patients whose BMI significantly decreased during study: {statistics_api.count(operation,  query_series)}")

print(f"Mean BMI of patients whose BMI significantly decreased: {statistics_api.mean(operation, query_series)} ")

Patients whose BMI decreased during study: 5
Patients whose BMI increased during study: 13
Patients whose BMI significantly decreased during study: 5
Mean BMI of patients whose BMI significantly decreased: 23.85334900613252 


# 6. Run Statistics Library

In [7]:
from smart_broker_api import statistics_api

type_distribution="normalunit"
type_ranking="cdf"
alternative = "two-sided"
print(statistics_api.count(operation,  series_1_id))
print(statistics_api.mean(operation,  series_1_id))
print(statistics_api.chisquare(operation,  series_1_id, series_2_id))
print(statistics_api.kolmogorov_smirnov_test(operation,  series_1_id, type_distribution))
print(statistics_api.kurtosis(operation,  series_1_id))
print(statistics_api.levene_test(operation,  series_1_id, series_2_id))
print(statistics_api.mann_whitney_u_test(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.min_max(operation,  series_1_id))
print(statistics_api.paired_t_test(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.pearson(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.skewness(operation,  series_1_id))
print(statistics_api.spearman(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.student_t_test(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.variance(operation,  series_1_id))
print(statistics_api.welch_t_test(operation,  series_1_id, series_2_id, alternative))
print(statistics_api.wilcoxon_signed_rank_test(operation,  series_1_id, series_2_id, alternative))

20
22.76433814224586
[380.0, 0.23583148842099294]
[0.95, 1.9073486328125338e-26]
-1.225866948308427
(0.003760306204864892, 0.9514246952891092)
(180.0, 0.5978625674290079)
(11.76951154694196, 30.172100539128525)
(-3.842534659324879, 0.00027445641167562454)
(0.9332515128886971, 1.9573960230445664e-09)
-0.08041628194553505
(0.9505828978811472, 1.394346860195128e-10)
(-0.999458438479299, 0.08097370675497725)
28.617281263722433
(-0.9994584384792992, 0.08097675209806848)
(0.9505828978811472, 1.394346860195128e-10)


In [9]:
??fig

Type:            Figure
String form:    
Figure({
    'data': [{'autobinx': False,
              'histnorm': 'probability density',
              'legendgroup': 'bmi_mean',
              'marker': {'color': 'rgb(31, 119, 180)'},
              'name': 'bmi_mean',
              'opacity': 0.7,
              'type': 'histogram',
              'x': [30.09579923988839, 29.435359597700078, 16.32648114674164,
                    20.068700736700052, 24.839296378896947, 27.831524855627176,
                    13.191912191736765, 18.525580056901987, 22.12142161608569,
                    27.71956696603433, 18.598758084216097, 29.051242656199765,
                    29.322578742343122, 14.830028026936311, 20.75641241204263,
                    22.9950964963041, 22.341160366997187, 28.860474997107282,
                    18.10088929395246, 20.2744789825052],
              'xaxis': 'x',
              'xbins': {'end': 30.09579923988839, 'size': 2.0, 'start': 13.191912191736765},
              'yaxis

In [10]:
from smart_broker_api import visualization_api
import plotly.graph_objects as go

# TODO: broken
# fig = visualization_api.histogram(operation, series_1_id, 20)
# fig.show()

fig = visualization_api.kernel_density_estimation(operation, series_1_id, 2)
fig.show()